In [1]:
import glob
import pickle
import pathlib
import pandas as pd
import networkx as nx

from zipfile import ZipFile

In [2]:
rel_data_folder_path = pathlib.Path("./../../data")
transport_data_path = rel_data_folder_path.joinpath('transport_data')
city_network_graphs = rel_data_folder_path.joinpath('network_graphs').joinpath('graphs')
city_network_bones = rel_data_folder_path.joinpath('network_graphs').joinpath('nodes-edges')

city_zips = list(transport_data_path.glob('*.zip'))

In [3]:
for city_data_path in city_zips:
    city_zf = ZipFile(city_data_path)
    city_name = city_data_path.name.removesuffix(".zip")
    
    city_nodes_df = pd.read_csv(city_zf.open(city_name + '/network_nodes.csv'),sep=";")
    city_zf.extract(city_name + '/network_nodes.csv', path=city_network_bones)
    
    city_network_df = pd.read_csv(city_zf.open(city_name + '/network_combined.csv'),sep=";")
    city_zf.extract(city_name + '/network_combined.csv', path=city_network_bones)
    
    city_graph = nx.MultiDiGraph()

    # Add nodes to the graph
    for _, row in city_nodes_df.iterrows():
        node_id = row['stop_I']
        city_graph.add_node(node_id, **row[1:].to_dict())

    # Add edges to the graph
    for _, row in city_network_df.iterrows():
        source = row['from_stop_I']
        target = row['to_stop_I']
        edge_data = row[2:].to_dict()
        city_graph.add_edge(source, target, **edge_data)
        
    with open(city_network_graphs.joinpath(city_name + '.gpickle'), 'wb') as f:
        pickle.dump(city_graph, f, pickle.HIGHEST_PROTOCOL)